In [18]:
from datetime import datetime, timedelta
import pandas as pd
import sqlite3
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import matplotlib.pyplot as plt
import ta
import yfinance as yf
import seaborn as sns
import numpy as np
from queue import Queue
import holidays
import scipy as sp
from itertools import combinations
from pymongo import MongoClient
import swifter

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

In [19]:
from helper.client import *
from helper.machinelearner import *
from helper.dbmgr import *
from helper.dataprocessor import *

# Create an instance of CoinbaseClient
client = CoinbaseClient(api_key, api_secret)

In [20]:
db_uri = 'mongodb://localhost:27017/'  
db_name = 'crypto_data'  

# Get and format data from CB

In [21]:
import pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["mydatabase"]
mycol = mydb["customers"]
mydict = { "name": "John", "address": "Highway 37" }

x = mycol.insert_one(mydict)

print(x.inserted_id)

65e8334ff203e96a0163b6b2


In [22]:
# unique_pids = client.get_unique_product_ids()
# unique_pids = [f"{i}-{'USDC'}" for i in unique_pids]

# # start_time = datetime(2022, 10, 1)
# start_time = datetime(2024, 1, 1)
# end_time = datetime(2024, 2, 29)

# db_manager = DatabaseManager(start_time=start_time, end_time=end_time)
# print(db_manager.check_row_counts())

# db_manager.main(unique_pids)

# Technical Indicator creation

In [23]:
# fe = FeatureEngineer(db_name)
# processed_data = fe.run()

# Preprocessing creation
- Labels
- Scaling
- Normalisation
- Stationary features

In [24]:
# data_preparator = DataPreparator(db_name)

# # Prepare the data
# final_data, non_stationary_columns = data_preparator.prepare_data()

In [25]:
label_generator = LabelGenerator(
    mongo_uri='mongodb://localhost:27017/',
    db_name='crypto_data',
    account_value=5000,
    tx_fee=0.006,
    profit_level=0.003
)

# Now, call the run method to start the label generation process
label_generator.run()

# Machine Learning on one set of parameters
- Grid Search for best parameters
- view feature importance on the best set and trim

In [ ]:
mongo_uri = 'mongodb://localhost:27017/'  # Adjust as necessary
db_name = 'crypto_data'  # Your database name
collection_name = 'labeled_crypto_data'  # Your collection name
target_column = 'target15'  # Your target column name

trainer = MongoDBMLModelTrainer(mongo_uri, db_name, collection_name, target_column)
trainer.train_and_evaluate()

# Save best model for backtesting

# Backtester